# Dense Retrieval

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
 !pip install cohere 
 !pip install weaviate-client Annoy

  Running setup.py clean for Annoy
Failed to build Annoy
    Running setup.py install for Annoy: started
    Running setup.py install for Annoy: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\nigus\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\nigus\\AppData\\Local\\Temp\\pip-install-7qvumyog\\annoy_bea17530ca9046cf85a911ad66547c51\\setup.py'"'"'; __file__='"'"'C:\\Users\\nigus\\AppData\\Local\\Temp\\pip-install-7qvumyog\\annoy_bea17530ca9046cf85a911ad66547c51\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\nigus\AppData\Local\Temp\pip-wheel-z7pt7al0'
       cwd: C:\Users\nigus\AppData\Local\Temp\pip-install-7qvumyog\annoy_bea17530ca9046cf85a911ad66547c51\
  Complete output (12 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating 

In [4]:
import cohere
co = cohere.Client('eXeH0uBrIn6lzRbrG7JyjHheoFVxQAGtLI7J1HnT')

In [5]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(
    api_key='ugFJXG6voNWkQ35qwSi13dwk5ba8kSxgfUZ2UEoI')

In [6]:
client = weaviate.Client(
    url='https://cohere-demo.weaviate.network/',
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": 'eXeH0uBrIn6lzRbrG7JyjHheoFVxQAGtLI7J1HnT',
    }
)
client.is_ready() #check if True

UnexpectedStatusCodeException: Meta endpoint! Unexpected status code: 401, with response body: {'code': 401, 'message': 'invalid api key, please provide a valid api key'}.

## Part 1: Vector Database for semantic Search

In [7]:
def dense_retrieval(query, 
                    results_lang='en', 
                    properties = ["text", "title", "url", "views", "lang", "_additional {distance}"],
                    num_results=5):

    nearText = {"concepts": [query]}
    
    # To filter by language
    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }
    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText) # here is the difference. instead of using bm25 for searching most common words
        .with_where(where_filter) # we use vector search which means finding the nearest data to the query
        .with_limit(num_results)
        .do()
    )

    result = response['data']['Get']['Articles']

    return result

In [8]:
from utils import print_result

ImportError: cannot import name 'print_result' from 'utils' (C:\Users\nigus\anaconda3\lib\site-packages\utils\__init__.py)

In [9]:
query = "Who wrote Hamlet?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

NameError: name 'client' is not defined

In [ ]:
query = "What is the capital of Canada?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

In [ ]:
from utils import keyword_search

query = "What is the capital of Canada?"
keyword_search_results = keyword_search(query, client)
print_result(keyword_search_results)

### Complicated Query

In [ ]:
from utils import keyword_search

query = "Tallest person in history?"
keyword_search_results = keyword_search(query, client)
print_result(keyword_search_results)

In [ ]:
query = "Tallest person in history"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

In [ ]:
query = "أطول رجل في التاريخ"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

In [ ]:
query = "film about a time travel paradox"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

## Part 2: Building Semantic Search from Scratch

### Get the text archive:

In [10]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import re

ModuleNotFoundError: No module named 'annoy'

In [14]:
text = """""
"Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

### Chunking: 

In [11]:
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

NameError: name 'text' is not defined

In [ ]:
texts

In [ ]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [ ]:
title = 'Interstellar (film)'

texts = np.array([f"{title} {t}" for t in texts])

### Get the embeddings:

In [ ]:
response = co.embed(
    texts=texts.tolist()
).embeddings

In [ ]:
embeds = np.array(response)
embeds.shape

### Create the search index:

we create a kind of database for our query to store using annoyIndex function. it takes the number of the paragraph and idk the other and we will enter each embedings in the search index the build them finally save to `test.nn` file. then by embadding our query we can use `search_index.get_nns_by_vector` kind function to get the nearest results for our query

In [ ]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

In [ ]:
pd.set_option('display.max_colwidth', None)

def search(query):

  # Get the query's embedding
  query_embed = co.embed(texts=[query]).embeddings

  # Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)
  # Format the results
  results = pd.DataFrame(data={'texts': texts[similar_item_ids[0]],
                              'distance': similar_item_ids[1]})

  print(texts[similar_item_ids[0]])
    
  return results

In [ ]:
query = "How much did the film make?"
search(query)

# ANN vector search vs vector database

Aproximate nearest naighbor vector search libereries like annoy are easy to setup and they only store vectors

vector database search like weaviate store both text and vectors and they are easy to handle data update unlike ANN vector which we should recreate our dataindex again.